In [1]:
gamma_diffuse = 'gamma_20deg_0deg_run101___cta-prod3-lapalma3-2147m-LaPalma.simtel.gz'
sys_path = '/home/matteo/ctasoft/'
prepared_data_dir = 'prepared_data/output'

In [2]:
import sys
import os
sys.path.append(sys_path+"cta-benchmarks/Preparation/parameters_search")  # path contains python_file.py
from energy_bin_function import create_folders_energybin, which_energybin_folder

In [3]:
"""
script to find the optimized levels of cleaning for each image of an event.
Data are discriminated by the telescope type and by the energy of the event, then stored
in the appropriate .npz file in 'output/event_cam_energybin' folder.
This means that we have optimized cleaning levels for each type of telescope

"""

import numpy as np
from astropy import units as u

In [4]:
prepared_data_dir = '../../../' + prepared_data_dir + '/event_cam'

if not os.path.exists(prepared_data_dir):
    os.makedirs(prepared_data_dir, exist_ok=True)
    
output_folder = prepared_data_dir + '/../event_cam_energybin'

if not os.path.exists(output_folder):
    os.makedirs(output_folder, exist_ok=True)

In [5]:
output_folder = prepared_data_dir + '/../event_cam_energybin'

loaded = np.load(prepared_data_dir + '/event_' + gamma_diffuse + '.npz', allow_pickle=True)

data_loaded = loaded['data']

dict_folders = create_folders_energybin(output_folder)

# search the energy info and store in the right list the data
for row in data_loaded:
    if isinstance(row[1], type(u.Quantity(1, u.GeV))):
        energy_event = row[1]
        dest_folder = which_energybin_folder(energy_event)
    else:
        if row[0] == 'LSTCam':
            dict_folders[dest_folder]['LSTCam'].append(row)
        elif row[0] == 'NectarCam':
            dict_folders[dest_folder]['NectarCam'].append(row)
        elif row[0] == 'FlashCam':
            dict_folders[dest_folder]['FlashCam'].append(row)
        elif row[0] == 'SCTCam':
            dict_folders[dest_folder]['SCTCam'].append(row)

# storing the data
for key, value in dict_folders.items():
    for k, v in value.items():
        path = output_folder + f"/{key}/{k}"
        np.savez_compressed(path, data=v)